## Task 3: Map Matching

### 3.1 install FMM
#### 3.1.1 Install requirements
brew install boost gdal libomp expat bzip2
#### 3.1.2 Install C++ program and Python bindings
Build and install the program with cmake
Under the project folder
  mkdir build
  cd build
  cmake ..
  make -j4
  sudo make install
#### 3.1.3 Verfication of installation
Change to the parent folder which contains fmm_test.py
  cd ../example/python
  python fmm_test.py

#### FMM demo
https://github.com/cyang-kth/fmm/blob/master/example/notebook/fmm_example.ipynb

#### some bugs
find ~/anaconda3 -name libgdal.dylib
cmake -D GDAL_LIBRARY=/Users/yuwen/anaconda3/lib/libgdal.dylib ..

#### result
(base) yuwen@yuwendebijibendiannao build % fmm
[info][fmm_app_config.cpp:49 ] Start reading FMM configuration from arguments
fmm argument lists:
--ubodt (required) <string>: Ubodt file name
--network (required) <string>: Network file name
--network_id (optional) <string>: Network id name (id)
--source (optional) <string>: Network source name (source)
--target (optional) <string>: Network target name (target)
--gps (required) <string>: GPS file name
--gps_id (optional) <string>: GPS id name (id)
--gps_x (optional) <string>: GPS x name (x)
--gps_y (optional) <string>: GPS y name (y)
--gps_timestamp (optional) <string>: GPS timestamp name (timestamp)
--gps_geom (optional) <string>: GPS geometry name (geom)
--gps_point (optional): if specified read input data as gps point, otherwise (default) read input data as trajectory
--output (required) <string>: Output file name
--output_fields (optional) <string>: Output fields
  opath,cpath,tpath,mgeom,pgeom,
  offset,error,spdist,tp,ep,length,duration,speed,all
-k/--candidates (optional) <int>: Number of candidates (8)
-r/--radius (optional) <double>: search radius (network data unit) (300)
-e/--error (optional) <double>: GPS error (network data unit) (50)
--reverse_tolerance (optional) <double>: proportion of reverse movement allowed on an edge
-l/--log_level (optional) <int>: log level (2)
-s/--step (optional) <int>: progress report step (100)
--use_omp: use OpenMP for multithreaded map matching
-h/--help:print help information
For xml configuration, check example folder


In [ ]:
import os
import csv
from fmm import FastMapMatch, Network, NetworkGraph, UBODTGenAlgorithm, UBODT, FastMapMatchConfig
from tqdm import tqdm

: 

In [1]:

def fmm_map_matching(network_path, ubodt_path, input_csv, output_csv, k=6, radius=0.05, gps_error=0.0002, regenerate_ubodt=False, threshold=0.02):
    # Check if all files exist
    if not os.path.exists(network_path):
        print(f"Network file {network_path} does not exist.")
        return
    
    if not os.path.exists(input_csv):
        print(f"Input CSV file {input_csv} does not exist.")
        return
    
    if regenerate_ubodt and os.path.exists(ubodt_path):
        os.remove(ubodt_path)

    # Load the network
    network = Network(network_path, "fid", "u", "v")
    print(f"Nodes {network.get_node_count()} edges {network.get_edge_count()}")
    graph = NetworkGraph(network)

    # Generate UBODT if not exists or if regeneration is forced
    if not os.path.exists(ubodt_path) or regenerate_ubodt:
        ubodt_gen = UBODTGenAlgorithm(network, graph)
        status = ubodt_gen.generate_ubodt(ubodt_path, threshold, binary=False, use_omp=True)
        if not status:
            print("Error generating UBODT.")
            return
        print("UBODT generated successfully.")

    # Load UBODT
    ubodt = UBODT.read_ubodt_csv(ubodt_path)

    # Create FMM model
    model = FastMapMatch(network, graph, ubodt)
    fmm_config = FastMapMatchConfig(k, radius, gps_error)

    # Process the input CSV and write results to output CSV
    with open(input_csv, "r") as in_csv, open(output_csv, "w", newline='') as out_csv:
        reader = csv.reader(in_csv)
        writer = csv.writer(out_csv)
        
        # Skip header in input and write header in output
        next(reader)

        # TODO
        writer.writerow(["Index", "cpath", "mgeom", "opath", "offset", "length", "spdist"])

        for index, row in tqdm(enumerate(reader)):
            gps = eval(row[9])
            wkt = 'LINESTRING(' + ','.join([' '.join([str(j) for j in i]) for i in gps]) + ')'
            result = model.match_wkt(wkt, fmm_config)

            candidates = list(result.candidates)
            writer.writerow([
                index,
                str(list(result.cpath)),
                result.mgeom.export_wkt(),
                str(list(result.opath)),
                str([c.offset for c in candidates]),
                str([c.length for c in candidates]),
                str([c.spdist for c in candidates])
            ])


In [ ]:
fmm_map_matching(
        network_path="data/porto/edges.shp",
        ubodt_path="data/ubodt.txt",
        input_csv="data/train-1500.csv",
        output_csv="data/matched_routines.csv"
    )

In [1]:
import pandas as pd
file_path = 'data/matched_routines.csv'
df = pd.read_csv(file_path, nrows = 5)
df.head(5)


,Index,cpath,mgeom,opath,offset,length,spdist
0,0,"[1055, 4246, 1053, 683, 10920, 136005, 10918, ...","LINESTRING(-8.6186233 41.141456,-8.6183463 41....","[1055, 1055, 4246, 4246, 1053, 1053, 683, 683,...","[0.005765244498603186, 0.0059112713802073075, ...","[0.007070636992543766, 0.007070636992543766, 0...","[0.0, 0.0001460268816041218, 0.004467393434995..."
1,1,"[37970, 179985, 163124, 163131, 163127, 6677, ...","LINESTRING(-8.6398592 41.159752,-8.6400962 41....","[37970, 37970, 6677, 40590, 740, 40554, 40566,...","[0.0016796833541194568, 0.002184336142457972, ...","[0.0023412979739263237, 0.0023412979739263237,...","[0.0, 0.0005046527883385151, 0.001860062043703..."
2,2,[],LINESTRING(),[],[],[],[]
3,3,[],LINESTRING(),[],[],[],[]
4,4,"[109655, 157789, 109656, 157794, 7926, 7929, 7...","LINESTRING(-8.6458216 41.180406,-8.645788 41.1...","[109655, 109655, 157789, 7929, 7930, 133654, 7...","[0.00044984402869269397, 0.0004938143669138266...","[0.000586322276888652, 0.000586322276888652, 0...","[0.0, 4.397033822113258e-05, 0.000646106566636..."


In [2]:
with open('data/mr.csv', 'w',  encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    
    for th, i in enumerate(trajectories_mapmatched):
        trajectory_ID = i - 1
        ####gps point load needed
        raw_gps_trajectories = raw_gps_points[trajectory_ID]
        matched_success = "True"
        node_indice_sequence = nodes_route_list[th]
        edge_indice_sequence = edges_route_list[th]
        road_indice_sequence = roads_route_list[th]
        wsg84_points_sequence = str(wsg84_routes_list[th])
        if len(wsg84_points_sequence) > 32758:
            print("warning long wsg84 route detected! Splitting data into 2 cells for trajectory ID", i)
            #data = [trajectory_ID, raw_gps_trajectories, matched_success, node_indice_sequence, edge_indice_sequence, 
                    #road_indice_sequence, str(wsg84_points_sequence[0:32758]), str(wsg84_points_sequence[32758:])]
            data = [trajectory_ID, raw_gps_trajectories, matched_success, node_indice_sequence, edge_indice_sequence, 
                    str(wsg84_points_sequence[0:32758]), str(wsg84_points_sequence[32758:])]
        else:
            #data = [trajectory_ID, raw_gps_trajectories, matched_success, node_indice_sequence, edge_indice_sequence, 
                    #road_indice_sequence, str(wsg84_points_sequence), ""]
            data = [trajectory_ID, raw_gps_trajectories, matched_success, node_indice_sequence, edge_indice_sequence, 
                    str(wsg84_points_sequence[0:32758]), str(wsg84_points_sequence[32758:])]
        writer.writerow(data)
            
    for th, i in enumerate(trajectories_failed):
        trajectory_ID = i - 1
        raw_gps_trajectories = raw_gps_points[trajectory_ID]
        matched_success = "FALSE"
        data = [trajectory_ID, raw_gps_trajectories, matched_success, "", "", "", ""]
        writer.writerow(data)

NameError: name 'csv' is not defined